#### CNN1: Convolutional Layers, Pooling Layers and Fully Connected Layers

In [1]:
# load packages
import os
import requests
import zipfile
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

In [2]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
# Functions
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [4]:
# Custom Dataset Class
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [5]:
def download_and_extract_data(data_url, data_zip_path, data_folder):
    if not os.path.isfile(data_zip_path):
        print('Downloading data...')
        response = requests.get(data_url, stream=True)
        with open(data_zip_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print('Data download completed.')

        # Extract data.zip
        print('Extracting data...')
        with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print('Data extraction completed.')
    else:
        print('data.zip already exists.')
        # Check if the extracted folder exists
        if not os.path.exists(data_folder):
            print('Extracting data...')
            with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
                zip_ref.extractall('.')
            print('Data extraction completed.')
        else:
            print('Data already extracted.')

# Set data URL and paths
data_url = 'https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip'
data_zip_path = 'data.zip'
data_folder = 'data'  # Extracted folder name

# Download and extract data
download_and_extract_data(data_url, data_zip_path, data_folder)

data.zip already exists.
Extracting data...
Data extraction completed.


In [6]:
# Load training and validation data
dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

# Load test data
dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

# Print data shapes
print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 139587)


In [7]:
# Creating Dataset Instances and Data Loaders

batch_size = 64
dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

torch.Size([203701, 1, 100, 40]) torch.Size([203701])


In [8]:
# Viewing dataset_train
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[0.3626, 0.0018, 0.3622,  ..., 0.0073, 0.3604, 0.0028],
          [0.3626, 0.0018, 0.3622,  ..., 0.0123, 0.3604, 0.0028],
          [0.3626, 0.0018, 0.3622,  ..., 0.0073, 0.3604, 0.0028],
          ...,
          [0.3625, 0.0041, 0.3621,  ..., 0.0123, 0.3600, 0.0030],
          [0.3625, 0.0041, 0.3620,  ..., 0.0123, 0.3600, 0.0030],
          [0.3625, 0.0041, 0.3620,  ..., 0.0123, 0.3599, 0.0125]]]],
       dtype=torch.float64)
tensor([2.], dtype=torch.float64)
torch.Size([1, 1, 100, 40]) torch.Size([1])


In [9]:
class CNN1(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        n_features = input_dim[2]

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(4, n_features), padding=(3, 0), dilation=(2, 1))
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=4)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=2)
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=2)

        # Max pooling layers
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)

        # Fully connected layers
        self.fc1 = nn.Linear(26 * 32, 32)
        self.fc2 = nn.Linear(32, output_dim)

        # Activation function
        self.relu = nn.LeakyReLU()

    def forward(self, x):
        # Apply convolutional layers with activation
        out = self.relu(self.conv1(x))                  # [batch_size, 16, H_out, W_out]
        out = out.permute(0, 1, 3, 2).reshape(out.size(0), out.size(1), -1)  # [batch_size, 16, W_out * H_out]

        out = self.relu(self.conv2(out))                # [batch_size, 16, L_out]
        out = self.maxpool1(out)                        # [batch_size, 16, L_out//2]

        out = self.relu(self.conv3(out))                # [batch_size, 32, L_out//2]
        out = self.relu(self.conv4(out))                # [batch_size, 32, L_out//2]
        out = self.maxpool2(out)                        # [batch_size, 32, L_out//4]

        # Flatten
        out = out.view(out.size(0), -1)                 # [batch_size, 32 * (L_out//4)]

        # Fully connected layers
        out = self.relu(self.fc1(out))                  # [batch_size, 32]
        out = self.fc2(out)                             # [batch_size, output_dim]

        return out

In [10]:
input_dim = (1, 100, 40)  
output_dim = dataset_train.num_classes  
Model_CNN1 = CNN1(input_dim=input_dim, output_dim=output_dim)
summary(Model_CNN1, input_size=(1, 1, 100, 40))

Layer (type:depth-idx)                   Output Shape              Param #
CNN1                                     [1, 3]                    --
├─Conv2d: 1-1                            [1, 16, 100, 1]           2,576
├─LeakyReLU: 1-2                         [1, 16, 100, 1]           --
├─Conv1d: 1-3                            [1, 16, 97]               1,040
├─LeakyReLU: 1-4                         [1, 16, 97]               --
├─MaxPool1d: 1-5                         [1, 16, 48]               --
├─Conv1d: 1-6                            [1, 32, 50]               1,568
├─LeakyReLU: 1-7                         [1, 32, 50]               --
├─Conv1d: 1-8                            [1, 32, 52]               3,104
├─LeakyReLU: 1-9                         [1, 32, 52]               --
├─MaxPool1d: 1-10                        [1, 32, 26]               --
├─Linear: 1-11                           [1, 32]                   26,656
├─LeakyReLU: 1-12                        [1, 32]                   --

In [11]:
# setting up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Model_CNN1.parameters(), lr=0.0001)

In [12]:
# A function to encapsulate the training loop
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
    
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            # move data to GPU
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
            # print("inputs.shape:", inputs.shape)
            # zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass
            # print("about to get model output")
            outputs = model(inputs)
            # print("done getting model output")
            # print("outputs.shape:", outputs.shape, "targets.shape:", targets.shape)
            loss = criterion(outputs, targets)
            # Backward and optimize
            # print("about to optimize")
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            torch.save(model, './best_val_model_CNN1_pytorch')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')

    return train_losses, test_losses

In [13]:
train_losses_CNN1, val_losses_CNN1 = batch_gd(Model_CNN1, criterion, optimizer, 
                                    train_loader, val_loader, epochs=50)

  2%|█▋                                                                                 | 1/50 [00:57<46:37, 57.09s/it]

model saved
Epoch 1/50, Train Loss: 1.0305,           Validation Loss: 1.0922, Duration: 0:00:57.087603, Best Val Epoch: 0


  4%|███▎                                                                               | 2/50 [01:32<35:21, 44.21s/it]

Epoch 2/50, Train Loss: 1.0201,           Validation Loss: 1.0929, Duration: 0:00:35.186950, Best Val Epoch: 0


  6%|████▉                                                                              | 3/50 [02:07<31:30, 40.23s/it]

...

Epoch 48/50, Train Loss: 0.6763,           Validation Loss: 0.9060, Duration: 0:00:35.207769, Best Val Epoch: 46


 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [29:13<00:35, 35.05s/it]

Epoch 49/50, Train Loss: 0.6742,           Validation Loss: 0.9157, Duration: 0:00:34.350896, Best Val Epoch: 46


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [29:48<00:00, 35.77s/it]

Epoch 50/50, Train Loss: 0.6717,           Validation Loss: 0.8868, Duration: 0:00:34.585625, Best Val Epoch: 46


In [14]:
import csv

# Save training losses
with open('train_losses_CNN1.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Train_Loss'])
    for epoch, loss in enumerate(train_losses_CNN1, start=1):
        writer.writerow([epoch, loss])

# Save validation losses
with open('val_losses_CNN1.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Epoch', 'Val_Loss'])
    for epoch, loss in enumerate(val_losses_CNN1, start=1):
        writer.writerow([epoch, loss])

print("Training and validation losses have been saved as CSV files.")

Training and validation losses have been saved as CSV files.


In [15]:
plt.figure(figsize=(15,6))
plt.plot(train_losses_CNN1, label='train loss - CNN1')
plt.plot(val_losses_CNN1, label='validation loss - CNN1')
plt.legend()

In [16]:
model = torch.load('best_val_model_CNN1_pytorch')

n_correct = 0.
n_total = 0.
for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    # update counts
    n_correct += (predictions == targets).sum().item()
    n_total += targets.shape[0]

test_acc = n_correct / n_total
print(f"Test acc: {test_acc:.4f}")

C:\Users\liujunyuan\AppData\Local\Temp\ipykernel_6868\1948593015.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('best_val_model_CNN1_pytorch')


Test acc: 0.6495


In [17]:
# model = torch.load('best_val_model_pytorch')
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions) 

In [18]:
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

accuracy_score: 0.6495038999770589
              precision    recall  f1-score   support

           0     0.6257    0.6428    0.6341     47915
           1     0.7507    0.6341    0.6875     48050
           2     0.5904    0.6740    0.6294     43523

    accuracy                         0.6495    139488
   macro avg     0.6556    0.6503    0.6503    139488
weighted avg     0.6577    0.6495    0.6510    139488

